###  ISA

In [28]:
import numpy as np

deg2rad = np.pi/180
grav = 9.81
R = 287.05          # Gas constant (Nm/kgK)

def rho(h):
    lr = -6.5/1000      # gradient de température (°K/m)
    T0 = 273.15 + 15
    rho0 = 1.225
    if h <= 11000 :
        T = T0 + lr * h
        rho = rho0 * (T/T0)**(-(1 + grav / (lr * R)))
    else :
        rho = rho0 * ((T0 + lr * 11000)/T0)**(-(1 + grav / (lr * R))) * np.exp(-grav * (h - 11000) / (R * (T0 + lr * 11000)))
    return rho

### AIRBUS A300 A/C Data



In [29]:
S_ref = 240
l_ref = 6.5
hG = 0.5 
Ixx = 5.88e6
Izz = 11.1e6
Ixz = 0

M = 120000 

Cx0 = 0.02
ki = 0.06
Cz_alpha = 5.0
alpha0 = -2.0 * deg2rad

Cy_beta = -0.65
Cy_dl = 0.0
Cy_dn = 0.19

Cl_beta = -0.92
Cl_p = -18.6
Cl_r = 5.89
Cl_dl = -0.56
Cl_dn = 0.13

Cn_beta_25 = 0.98
Cn_p = -1.37
Cn_r = -7.18
Cn_dl = -0.02
Cn_dn = -0.56


Cy_x = np.array([Cy_beta, 0, 0, Cy_dl, Cy_dn])
Cl_x = np.array([Cl_beta, Cl_p, Cl_r, Cl_dl, Cl_dn])
Cn_beta = Cn_beta_25 + (hG - 0.25) * Cy_beta
Cn_x = np.array([Cn_beta, Cn_p, Cn_r, Cn_dl, Cn_dn])

D = 1 - Ixz*Ixz / Ixx*Izz
Cl_x_tilde = (Cl_x + (Ixz/Izz) * Cn_x) / D
Cn_x_tilde = (Cn_x + (Ixz/Ixx) * Cl_x) / D

print(Cl_x_tilde, Cl_x)
print(Cn_x_tilde, Cn_x)

[ -0.92 -18.6    5.89  -0.56   0.13] [ -0.92 -18.6    5.89  -0.56   0.13]
[ 0.8175 -1.37   -7.18   -0.02   -0.56  ] [ 0.8175 -1.37   -7.18   -0.02   -0.56  ]


### Longitudinal Equilibrium

In [30]:
h = 0
V = 125

rho = rho(h)
print(f'rho = {rho:.3f}')

q_dyn = 0.5 * rho * V**2

Cz = M * grav / (q_dyn * S_ref)
Cx = Cx0 + ki * Cz**2
f = Cz / Cx
print(f'Cz = {Cz:.3f}, Cx = {Cx:.3f}, finesse = {f:.3f}')

alpha = Cz / Cz_alpha + alpha0
print(f'alpha = {alpha/deg2rad:.2f} deg')

Thrust = M * grav / f 
print(f'Thrust = {Thrust/1000:.2f} kN')


rho = 1.225
Cz = 0.513, Cx = 0.036, finesse = 14.332
alpha = 3.87 deg
Thrust = 82.14 kN


In [31]:
def dyn(x):
    beta, r, p, phi, dl, dn = x
    theta = alpha * np.cos(phi) + beta * np.sin(phi)
    Cy = Cy_x @ np.array([beta, p, r, dl, dn])
    Cl = Cl_x_tilde @ (np.array([beta, p, r, dl, dn]) * np.array([1, (l_ref / V), (l_ref / V), 1, 1]))
    Cn = Cn_x_tilde @ (np.array([beta, p, r, dl, dn]) * np.array([1, (l_ref / V), (l_ref / V), 1, 1]))
    d_beta = -r * np.cos(alpha) + p * np.sin(alpha) + (q_dyn * S_ref * Cy + M * grav * np.cos(theta) * np.sin(phi))/ (M * V * np.cos(beta))
    d_r = q_dyn * S_ref * l_ref * Cn / Izz
    d_p = q_dyn * S_ref * l_ref * Cl / Ixx
    d_phi = p + r * np.cos(phi) * np.tan(theta)
    return np.array([d_beta, d_r, d_p, d_phi])

def jac(V, alpha, rho):
    q_dyn = 0.5 * rho * V**2
    y_x = q_dyn * S_ref * Cy_x / (M * V)
    l_x = q_dyn * S_ref * l_ref * (Cl_x_tilde * np.array([1, (l_ref / V), (l_ref / V), 1, 1])) / Ixx 
    n_x = q_dyn * S_ref * l_ref * (Cn_x_tilde * np.array([1, (l_ref / V), (l_ref / V), 1, 1])) / Izz
    theta = alpha
    A = np.array([[y_x[0], -np.cos(alpha), np.sin(alpha), grav * np.cos(theta) / V],
                  [n_x[0], n_x[2], n_x[1], 0],
                  [l_x[0], l_x[2], l_x[1], 0],
                  [0, np.tan(theta), 1, 0]])
    B = np.array([[y_x[3], y_x[4]],
                  [n_x[3], n_x[4]],
                  [l_x[3], l_x[4]],
                  [0, 0]])
    return A, B


In [32]:
### coordinate turn
from scipy.optimize import fsolve

phi = 10 * deg2rad
beta = 0

def func(y):
    r, p, dl, dn = y
    x = np.array([beta, r, p, phi, dl, dn])
    return dyn(x)

r, p, dl, dn = fsolve(func, [0, 0, 0, 0]) / deg2rad

print(f'r = {r:.2f}°/s, p = {p:.2f}°/s')
print(f'dl = {dl:.2f}°, dn = {dn:.2f}°')


r = 0.76°/s, p = -0.05°/s
dl = 0.38°, dn = -0.52°


In [33]:
### turn with dl only
from scipy.optimize import fsolve

phi = 10 * deg2rad
dn = 0

def func(y):
    r, p, dl, beta = y
    x = np.array([beta, r, p, phi, dl, dn])
    return dyn(x)

r, p, dl, beta = fsolve(func, [0, 0, 0, 0]) / deg2rad

print(f'r = {r:.2f}°/s, p = {p:.2f}°/s')
print(f'dl = {dl:.2f}°, beta = {beta:.2f}°')

r = 0.74°/s, p = -0.05°/s
dl = -0.06°, beta = 0.33°


In [34]:
### turn with dn (rudder) only
from scipy.optimize import fsolve

phi = 10 * deg2rad
dl = 0

def func(y):
    r, p, dn, beta = y
    x = np.array([beta, r, p, phi, dl, dn])
    return dyn(x)

r, p, dn, beta = fsolve(func, [0, 0, 0, 0]) / deg2rad

print(f'r = {r:.2f}°/s, p = {p:.2f}°/s')
print(f'dn = {dn:.2f}°, beta = {beta:.2f}°')

r = 0.75°/s, p = -0.05°/s
dn = -0.07°, beta = 0.29°


In [ ]:

print(f'Vitesse = {V:.2f} m/s, alpha = {alpha/deg2rad:.2f}°')
A, B = jac(V, alpha, rho)

vp = np.linalg.eig(A)[0]

print('\n --- ROULIS PUR ---')
periode = -1/vp[0].real 
print(f'période = {periode:.2f} s')

print('\n --- DUTCH ROLL ---')
lam = -vp[1].real
wn = vp[1].imag
w0 = np.sqrt(wn**2 + lam**2)
zeta = lam / w0
periode = 2*np.pi / wn
print(f'période = {periode:.2f} s')
print(f'amortissment = {zeta:.2f}')

print('\n --- MODE SPIRAL ---')
periode = -1/vp[3].real 
print(f'période = {periode:.2f} s')



Vitesse = 125.00 m/s, alpha = 3.87°
[-2.43088277+0.j        -0.31185158+1.1111779j -0.31185158-1.1111779j
 -0.00290208+0.j       ]

 --- ROULIS PUR ---
période = 0.41 s

 --- DUTCH ROLL ---
période = 5.65 s
amortissment = 0.27

 --- MODE SPIRAL ---
période = 344.58 s


In [ ]:
### CLOSE LOOP

k_beta = 0
k_r= 1

K = np.array([[0, 0, 0, 0], 
              [k_beta, k_r, 0, 0]])

A_BF = A + B@K

vp = np.linalg.eig(A_BF)[0]

print('\n --- ROULIS PUR ---')
periode = -1/vp[0].real 
print(f'période = {periode:.2f} s')

print('\n --- DUTCH ROLL ---')
lam = -vp[1].real
wn = vp[1].imag
w0 = np.sqrt(wn**2 + lam**2)
zeta = lam / w0
periode = 2*np.pi / wn
print(f'période = {periode:.2f} s')
print(f'amortissment = {zeta:.2f}')

print('\n --- MODE SPIRAL ---')
periode = -1/vp[3].real 
print(f'période = {periode:.2f} s')


[-2.41151904+0.j         -0.68145582+0.94517141j -0.68145582-0.94517141j
 -0.03626679+0.j        ]

 --- ROULIS PUR ---
période = 0.41 s

 --- DUTCH ROLL ---
période = 6.65 s
amortissment = 0.58

 --- MODE SPIRAL ---
période = 27.57 s
